Generating final model to be used in the application with the optimized parameters from the optimization.

In [ ]:
chosen_model = 

Model persistence. The model can be loaded from the generated pickle file to be used in running python processes without the need of retraining it everytime.

In [ ]:
from joblib import dump

dump(chosen_model, './persistence/classifier_model.joblib')

# Can be loaded with:
# from joblib import load

# chosen_model = gnb
# classifier_model = ('./persistence/classifier_model.joblib')
# predictions = classifier_model.predict(url_list)